In [2]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [57]:
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("us_census_no_error.csv")

In [61]:
df.show()

+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|State FIPS|  State|116th Congressional District|2017 NAICS Code|   NAICS Description|Number of Establishments|Employment|Employment Noise Flag|1st Quarter Payroll ($1,000)|1st Quarter Payroll Noise Flag|Annual Payroll ($1,000)|Annual Payroll Noise Flag|
+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|         1|Alabama|                           1|         ------|Total for all sec...|                  15,921|  2,44,615|                    G|                   24,92,282|                             G|            1,04,17,508|       

In [60]:
from pyspark.sql.functions import udf
commaToDot = udf(lambda x : float(str(x).replace(',', '.')), FloatType())
df2=df.withColumn('Employment',commaToDot(df.Employment))

In [40]:
df.drop("2017 NAICS Code").columns

['State FIPS',
 'State',
 '116th Congressional District',
 'NAICS Description',
 'Number of Establishments',
 'Employment',
 'Employment Noise Flag',
 '1st Quarter Payroll ($1,000)',
 '1st Quarter Payroll Noise Flag',
 'Annual Payroll ($1,000)',
 'Annual Payroll Noise Flag']

In [54]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import FloatType
df = (
         df
         .withColumn('Employment', regexp_replace('Employment', ',', ''))
        .withColumn('Number of Establishments', regexp_replace('Number of Establishments', ',', ''))
        .withColumn('Annual Payroll ($1,000)', regexp_replace('Annual Payroll ($1,000)', ',', ''))
        .withColumn('1st Quarter Payroll ($1,000)', regexp_replace('1st Quarter Payroll ($1,000)', ',', ''))
#          .withColumn('Employment', df['Employment'].cast("int"))
     )

In [55]:
df.show()

+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|State FIPS|  State|116th Congressional District|2017 NAICS Code|   NAICS Description|Number of Establishments|Employment|Employment Noise Flag|1st Quarter Payroll ($1,000)|1st Quarter Payroll Noise Flag|Annual Payroll ($1,000)|Annual Payroll Noise Flag|
+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|         1|Alabama|                           1|         ------|Total for all sec...|                   15921|    244615|                    G|                     2492282|                             G|               10417508|       

In [74]:
df.dtypes

[('State FIPS', 'int'),
 ('State', 'string'),
 ('116th Congressional District', 'int'),
 ('2017 NAICS Code', 'string'),
 ('NAICS Description', 'string'),
 ('Number of Establishments', 'string'),
 ('Employment', 'int'),
 ('Employment Noise Flag', 'string'),
 ('1st Quarter Payroll ($1,000)', 'string'),
 ('1st Quarter Payroll Noise Flag', 'string'),
 ('Annual Payroll ($1,000)', 'string'),
 ('Annual Payroll Noise Flag', 'string')]

In [76]:
df.describe().show()

+-------+------------------+-------+----------------------------+---------------+--------------------+------------------------+------------------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|summary|        State FIPS|  State|116th Congressional District|2017 NAICS Code|   NAICS Description|Number of Establishments|        Employment|Employment Noise Flag|1st Quarter Payroll ($1,000)|1st Quarter Payroll Noise Flag|Annual Payroll ($1,000)|Annual Payroll Noise Flag|
+-------+------------------+-------+----------------------------+---------------+--------------------+------------------------+------------------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|  count|              9104|   9104|                        9104|           9104|                9104|                    9104|              1319|                 

In [73]:
from pyspark.sql.types import IntegerType
df = df.withColumn("Employment", df["Employment"].cast(IntegerType()))

In [98]:
import locale
# locale.setlocale(locale.LC_ALL, 'de_DE') #Germany
# df.Employment.astype(int).apply(locale.atof)
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
locale.atoi("9,0,0")

900

In [ ]:
def dict(x,y):
    return abs(x-y)
udf_dict = udf(dict, IntegerType())

In [89]:
df.select("Employment")

DataFrame[Employment: int]

In [94]:
df.select("Employment").show()


+----------+
|Employment|
+----------+
|      null|
|      null|
|       500|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
+----------+
only showing top 20 rows

